# Phase 1 - Setting everything up

Open this repository in a GitHub Codespace.
Before you start with anything else, make sure you setup the infrastructure required. Follow the readme file in the root folder to do this!

To start with Phase 1, if not already done run this in the top level folder:

```
pip install -r requirements.txt
```




In [8]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

Found Azure OpenAI API Base Endpoint: https://cog-bdxwqoioy5zpa.openai.azure.com/


To see if stuff works in general, you can run this.

In [9]:
response = client.chat.completions.create(
    model = model_name,    
    messages = [{"role" : "assistant", "content" : "The one thing I love more than anything else is "}],
)

print(response.choices[0].message.content)

the feeling of being completely immersed in a good story. It's like escaping into another world where anything is possible, and it can be through books, movies, music, or conversations with friends. That sense of connection and adventure is unmatched. What about you? What's something you love?


This is the object model for receiving questions.

In [10]:
from enum import Enum
from pydantic import BaseModel

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.



In [35]:

import re
from word2number import w2n


def extract_number(sentence: str) -> str:

    # Extract the word representing the number
    number_word = re.search(r'\b(one|two|three|four|five|six|seven|eight|nine|ten)\b', sentence, re.IGNORECASE)

    if number_word:
        # Convert the word to an integer
        number_int = w2n.word_to_num(number_word.group())
        return str(number_int)
    else:
        print("No number word found in the sentence.")
    

async def ask_question(ask: Ask):
    # """
    # # Ask a question
    # """

    # Send a completion call to generate an answer
    print('Sending a request to openai')
    
    start_phrase =  ask.question
    response: openai.types.chat.chat_completion.ChatCompletion = None
    
    response = client.chat.completions.create(
        model = deployment_name,
        messages = [{"role" : "assistant", "content" : start_phrase}, 
                     { "role" : "system", "content" : "Answer this question:"}]
    )

    print(response.choices[0].message.content)
    print(response)
    answer_string = response.choices[0].message.content
    if ask.type == QuestionType.multiple_choice:
        answer_string = answer_string[3:]
    elif ask.type == QuestionType.true_or_false:
        answer_string = answer_string.lower()
    elif ask.type == QuestionType.estimation:
        answer_string = extract_number(answer_string)
    answer = Answer(answer=answer_string)
    answer.correlationToken = ask.correlationToken
    answer.promptTokensUsed = response.usage.prompt_tokens
    answer.completionTokensUsed = response.usage.completion_tokens

    return answer

Use this snippet to try your method with several questions.

In [34]:

ask = Ask(question="How many movies are there in 'The Lord of the Rings' trilogy directed by Peter Jackson?", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer)


Sending a request to openai
There are three movies in "The Lord of the Rings" trilogy directed by Peter Jackson:

1. "The Lord of the Rings: The Fellowship of the Ring" (2001)
2. "The Lord of the Rings: The Two Towers" (2002)
3. "The Lord of the Rings: The Return of the King" (2003)
ChatCompletion(id='chatcmpl-A8OdCRZoK6B1p651Uhh6vzun5Z4rl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='There are three movies in "The Lord of the Rings" trilogy directed by Peter Jackson:\n\n1. "The Lord of the Rings: The Fellowship of the Ring" (2001)\n2. "The Lord of the Rings: The Two Towers" (2002)\n3. "The Lord of the Rings: The Return of the King" (2003)', refusal=None, role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'f

# Sample Questions
Sample Questions could look like this. Make sure your answer exactly  matches the required answer.



{
    "id": 3,
    "phase": 1,
    "question": "Which movie features a plot where a young girl named Dorothy is transported to a magical land via a tornado? 1) Cinderella 2) The Wizard of Oz 3) Alice in Wonderland 4) The Little Mermaid",
    "answer": "The Wizard of Oz",
    "type": "multiple_choice"
},
{
    "id": 4,
    "phase": 1,
    "question": "Is Yoda a character from the Star Trek universe: True or False?",
    "answer": false,
    "type": "true_or_false"
},
{
    "id": 5,
    "phase": 1,
    "question": "How many movies are there in 'The Lord of the Rings' trilogy directed by Peter Jackson?",
    "answer": 3,
    "type": "estimation"
}

Make sure you transfer your code changes into main.py (or additional files). 
You can test your app locally using uvicorn. (See Readme.md for details.)

Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase1 $AZURE_ENV_NAME
```
